# 1. Load CATPCHA images and get the transformed (X,y) data

In [2]:
import torch

from helper_functions.data_transformation import get_transformed_data_v2, get_transformed_data_for_captcha_evaluation

train_folder = "../data/train/combine"
test_folder = "../data/test/combine"
cached_data_path = "../data/prepared_data_cnn_v2.pth"

try:
    # Load the prepared data from cache if it already exists
    X_train, y_train, X_test, y_test, X_test_captcha, y_test_captcha, failed_segmentation_info = torch.load(cached_data_path)
    print("Loaded prepared data from cache.")

except FileNotFoundError:
    # Else prepare the data and save it
    X_train, y_train = get_transformed_data_v2(train_folder, is_train=True)
    X_test, y_test = get_transformed_data_v2(test_folder, is_train=False)
    X_test_captcha, y_test_captcha, failed_segmentation_info = get_transformed_data_for_captcha_evaluation(test_folder)

    torch.save((X_train, y_train, X_test, y_test, X_test_captcha, y_test_captcha, failed_segmentation_info), cached_data_path)
    print("Saved prepared data to cache.")


C:\Users\ongka\AppData\Local\Temp\ipykernel_26064\4202067423.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train, y_train, X_test, y_test, X_test_captcha, y_test_ca

Loaded prepared data from cache.


# 2. Instantiate RESNET Model


In [30]:
from models.resnet import ResNet50, CNNParams
from helper_functions.data_transformation import IMG_HEIGHT, IMG_WIDTH

# Train model
cached_model_path = "../trained_models/resnet50_v2_86_char_accuracy.pth"

resnet = None

cnn_params: CNNParams = {
    "num_classes": 36,
    "input_shape": (1, IMG_HEIGHT, IMG_WIDTH), # (channels, height, width)
    "learning_rate": 1.1*1e-3,
    "num_epochs": 20,
    "batch_size": 500,
}

try:
    # Load the trained model from cache if it already exists
    resnet = torch.load(cached_model_path)
    print("Loaded trained model from cache.")
except FileNotFoundError:
    # Else train the model and save it
    resnet = ResNet50(cnn_params)
    resnet.train_model(X_train, y_train)
    torch.save(resnet, cached_model_path)
    print("Saved trained model to cache.")


C:\Users\ongka\AppData\Local\Temp\ipykernel_26064\1076884564.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet = torch.load(cached_model_path)


Epoch [1/20], Loss: 0.8782
Epoch [2/20], Loss: 0.3884
Epoch [3/20], Loss: 0.3049
Epoch [4/20], Loss: 0.2501
Epoch [5/20], Loss: 0.2013
Epoch [6/20], Loss: 0.1791
Epoch [7/20], Loss: 0.1550
Epoch [8/20], Loss: 0.1347
Epoch [9/20], Loss: 0.1274
Epoch [10/20], Loss: 0.1186
Epoch [11/20], Loss: 0.0981
Epoch [12/20], Loss: 0.0944
Epoch [13/20], Loss: 0.0894
Epoch [14/20], Loss: 0.0779
Epoch [15/20], Loss: 0.0721
Epoch [16/20], Loss: 0.0742
Epoch [17/20], Loss: 0.0710
Epoch [18/20], Loss: 0.0610
Epoch [19/20], Loss: 0.0551
Epoch [20/20], Loss: 0.0465
Training complete!
Saved trained model to cache.


# 3. Evaluate model

In [31]:
import json
from helper_functions.evaluation import evaluate_character_performance, evaluate_captcha_performance

character_performance = evaluate_character_performance(resnet, X_test, y_test)
print(f"Character level performance (skipping failed segmentations): {json.dumps(character_performance, indent=2)}\n")

captcha_performance = evaluate_captcha_performance(resnet, X_test_captcha, y_test_captcha, failed_segmentation_info)

Character level performance (skipping failed segmentations): {
  "accuracy": 0.9115130299494361,
  "precision": 0.914033491952163,
  "recall": 0.9095050395408215,
  "f1_score": 0.9107453727646201
}

Captcha level performance: {
  "segmentation_accuracy": 0.9091869060190074,
  "captcha_accuracy (skipping SF)": 0.6387921022067363,
  "captcha_accuracy (including SF)": 0.5807814149947201,
  "character_accuracy (including SF)": 0.82663139329806
}

